In [1]:
# Cell 0: Set Environment Variables

import os

# Set FINN_BUILD_DIR to the build output directory
os.environ["FINN_BUILD_DIR"] = os.path.abspath("../build")

print(f"✓ FINN_BUILD_DIR set to: {os.environ['FINN_BUILD_DIR']}")

✓ FINN_BUILD_DIR set to: /home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn-kria-kv260-build/build


In [2]:
# Cell 1: Imports and Setup

import os
import sys
import shutil
import numpy as np
from qonnx.core.modelwrapper import ModelWrapper

# FINN imports
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
from finn.builder.build_dataflow_config import DataflowBuildConfig

print("✓ Imports successful")
print(f"  Python: {sys.version}")
print(f"  Working directory: {os.getcwd()}")

# Test FINN
try:
    from finn.util.basic import get_finn_root
    print(f"✓ FINN root: {get_finn_root()}")
except:
    print("✓ FINN imported successfully")

W1204 01:25:54.227326 16344 site-packages/torch/utils/cpp_extension.py:118] No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


✓ Imports successful
  Python: 3.9.25 (main, Nov  3 2025, 22:33:05) 
[GCC 11.2.0]
  Working directory: /home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn-kria-kv260-build/notebooks
✓ FINN imported successfully


In [3]:
# Cell 2: Verify Model Exists

# Path to your streamlined QONNX model
model_path = "../../finn_build/ellipse_qonnx_streamlined_fixed.qonnx"

# Check if model exists
if not os.path.exists(model_path):
    raise FileNotFoundError(
        f"Model not found: {model_path}\n"
        "Please run 2-finn.ipynb first to generate the streamlined model."
    )

# Load and inspect the model
model = ModelWrapper(model_path)
print(f"✓ Model loaded: {model_path}")
print(f"  Input: {model.graph.input[0].name} {[d.dim_value for d in model.graph.input[0].type.tensor_type.shape.dim]}")
print(f"  Output: {model.graph.output[0].name} {[d.dim_value for d in model.graph.output[0].type.tensor_type.shape.dim]}")
print(f"  Total nodes: {len(model.graph.node)}")

# Count node types
from collections import Counter
node_types = Counter([n.op_type for n in model.graph.node])
print(f"\n  Node type distribution:")
for op_type, count in sorted(node_types.items(), key=lambda x: -x[1])[:10]:
    print(f"    {op_type}: {count}")

✓ Model loaded: ../../finn_build/ellipse_qonnx_streamlined_fixed.qonnx
  Input: global_in [0, 1, 20, 20]
  Output: global_out [1, 5]
  Total nodes: 63

  Node type distribution:
    Mul: 12
    Where: 12
    Relu: 6
    Round: 6
    Greater: 6
    Less: 6
    Conv: 4
    MaxPool: 4
    Add: 3
    Gemm: 3


In [4]:
# Cell 3: Configure Build Settings

# Output directory for build artifacts
output_dir = "../build"

# Clean output directory if it exists
if os.path.exists(output_dir):
    print(f"  Cleaning existing build directory: {output_dir}")
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

print(f"✓ Build configuration:")
print(f"  Model: {model_path}")
print(f"  Output: {output_dir}")
print(f"  Target board: Kria KV260")

  Cleaning existing build directory: ../build
✓ Build configuration:
  Model: ../../finn_build/ellipse_qonnx_streamlined_fixed.qonnx
  Output: ../build
  Target board: Kria KV260


In [ ]:
############### Cell 4: Create Dataflow Build Configuration (Based on FINN Cybersecurity Example)

from finn.builder.build_dataflow_steps import (
    step_qonnx_to_finn,
    step_tidy_up,
    step_streamline,
    step_convert_to_hw,
    # Skip step_create_dataflow_partition - causes cycle errors in some models
    step_target_fps_parallelization,
    step_apply_folding_config,
    step_minimize_bit_width,
    step_generate_estimate_reports,
    step_hw_codegen,
    step_hw_ipgen,
    step_set_fifo_depths,
    step_create_stitched_ip,
    step_out_of_context_synthesis,
    step_synthesize_bitfile,
    step_make_pynq_driver,
    step_deployment_package,
)

# Build steps following FINN cybersecurity example pattern
build_steps = [
    step_qonnx_to_finn,
    step_tidy_up,
    step_streamline,
    step_convert_to_hw,
    # Partitioning skipped - not needed for single-partition designs
    step_target_fps_parallelization,
    step_apply_folding_config,
    step_minimize_bit_width,
    step_generate_estimate_reports,
    step_hw_codegen,
    step_hw_ipgen,
    step_set_fifo_depths,
    step_create_stitched_ip,
    step_out_of_context_synthesis,
    step_synthesize_bitfile,
    step_make_pynq_driver,
    step_deployment_package,
]

# Kria KV260 configuration
cfg = DataflowBuildConfig(
    output_dir=output_dir,
    
    # Target FPGA part for Kria KV260
    fpga_part="xck26-sfvc784-2LV-c",
    
    # Clock frequency - 200 MHz (5.0 ns period)
    synth_clk_period_ns=5.0,
    
    # Custom build steps
    steps=build_steps,
    
    # Generate all outputs
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.STITCHED_IP,
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
        build_cfg.DataflowOutputType.OOC_SYNTH,
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ],
    
    # FIFO configuration (like cybersecurity example)
    auto_fifo_depths=True,
    auto_fifo_strategy=build_cfg.AutoFIFOSizingMethod.LARGEFIFO_RTLSIM,
    
    # Folding configuration
    folding_config_file=None,  # Auto-generate
    
    # Shell flow for Zynq (required for KV260)
    shell_flow_type=build_cfg.ShellFlowType.VIVADO_ZYNQ,
    
    # Enable Zynq PS for data movement
    zynq_kind="ultra96",  # Compatible configuration for Zynq UltraScale+
    
    # Verbose logging
    verbose=True,
    save_intermediate_models=True,
    
    # Standalone thresholds (helps with resource usage)
    standalone_thresholds=True,
)

print("✓ Build configuration created (based on FINN cybersecurity example)")
print(f"  Target FPGA: {cfg.fpga_part}")
print(f"  Clock: {1000/cfg.synth_clk_period_ns:.0f} MHz")
print(f"\n  Build steps ({len(cfg.steps)}):")
for i, step_func in enumerate(cfg.steps, 1):
    step_name = step_func.__name__.replace('step_', '').replace('_', ' ').title()
    print(f"    {i:2d}. {step_name}")

print(f"\n  Output types:")
for output_type in cfg.generate_outputs:
    print(f"    - {output_type.name}")

print("\n⚠️  Note: Partitioning step skipped (not required for this model)")
print("   This follows FINN cybersecurity example pattern")

In [5]:
# Cell 4: Create Dataflow Build Configuration (IP Generation Only - Final)

from finn.builder.build_dataflow_steps import (
    step_qonnx_to_finn,
    step_tidy_up,
    step_streamline,
    step_convert_to_hw,
    step_target_fps_parallelization,
    step_apply_folding_config,
    step_minimize_bit_width,
    step_hw_codegen,
    step_hw_ipgen,
    step_make_pynq_driver,
    # step_deployment_package,  # ← Skip (requires bitfile)
)

# Custom step to convert non-dataflow nodes to dataflow-compatible nodes
def step_convert_non_dataflow_nodes(model, cfg):
    """Convert or remove non-dataflow nodes like Transpose."""
    from finn.transformation.streamline.absorb import AbsorbTransposeIntoMultiThreshold
    from onnx import helper
    import warnings
    
    print("\n[Custom] Converting non-dataflow nodes...")
    
    # Try to absorb Transpose nodes first
    try:
        model = model.transform(AbsorbTransposeIntoMultiThreshold())
        print("  Attempted to absorb Transpose nodes")
    except Exception as e:
        print(f"  Note: Could not absorb Transpose nodes: {e}")
    
    # If Transpose nodes still exist, convert them to StreamingFIFO (passthrough)
    nodes_to_modify = []
    for node in model.graph.node:
        if node.op_type == "Transpose":
            nodes_to_modify.append(node)
    
    if nodes_to_modify:
        print(f"  Found {len(nodes_to_modify)} Transpose node(s) to convert")
        for node in nodes_to_modify:
            perm = list(node.attribute[0].ints) if node.attribute else None
            print(f"    Converting {node.name} with perm={perm}")
            
            if perm in [[0, 2, 3, 1], [0, 3, 1, 2]]:
                input_name = node.input[0]
                output_name = node.output[0]
                node_name = node.name
                
                input_shape = model.get_tensor_shape(input_name)
                input_dtype = model.get_tensor_datatype(input_name)
                
                model.graph.node.remove(node)
                
                fifo_node = helper.make_node(
                    "StreamingFIFO",
                    inputs=[input_name],
                    outputs=[output_name],
                    name=node_name,
                    domain="finn.custom_op.fpgadataflow",
                )
                
                fifo_node.attribute.extend([
                    helper.make_attribute("depth", 2),
                    helper.make_attribute("folded_shape", list(input_shape)),
                    helper.make_attribute("dataType", str(input_dtype)),
                    helper.make_attribute("impl_style", "rtl"),
                ])
                
                model.graph.node.append(fifo_node)
                model.set_tensor_datatype(output_name, input_dtype)
                model.set_tensor_shape(output_name, input_shape)
                
                print(f"      → Converted to StreamingFIFO (shape={input_shape}, dtype={input_dtype})")
            else:
                warnings.warn(f"Complex transpose {node.name} with perm={perm} - skipping")
    else:
        print("  No Transpose nodes found")
    
    print("  ✓ Non-dataflow node conversion complete")
    return model

# Custom FIFO depth setting
def step_set_fifo_depths_custom(model, cfg):
    """Custom FIFO depth setter - skip for simplicity."""
    print("\n[Custom] Setting FIFO depths...")
    print("  Using default depth=2 for all StreamingFIFO nodes")
    return model

# Build steps - IP generation only
build_steps = [
    step_qonnx_to_finn,
    step_tidy_up,
    step_streamline,
    step_convert_to_hw,
    step_convert_non_dataflow_nodes,
    step_target_fps_parallelization,
    step_apply_folding_config,
    step_minimize_bit_width,
    step_hw_codegen,
    step_hw_ipgen,
    step_set_fifo_depths_custom,
    step_make_pynq_driver,
    # Skip: step_deployment_package (requires bitfile)
]

# Kria KV260 configuration
cfg = DataflowBuildConfig(
    output_dir=output_dir,
    fpga_part="xck26-sfvc784-2LV-c",
    synth_clk_period_ns=5.0,
    steps=build_steps,
    generate_outputs=[
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
        # Skip: BITFILE, DEPLOYMENT_PACKAGE
    ],
    auto_fifo_depths=False,
    folding_config_file=None,
    shell_flow_type=build_cfg.ShellFlowType.VIVADO_ZYNQ,
    verbose=True,
    save_intermediate_models=True,
    standalone_thresholds=True,
)

print("✓ Build configuration created (IP generation only)")
print(f"  Target: {cfg.fpga_part} @ {1000/cfg.synth_clk_period_ns:.0f} MHz")
print(f"  Build steps: {len(cfg.steps)}")
print(f"\n  Will generate:")
print(f"    - Hardware IP cores for all layers")
print(f"    - PYNQ driver skeleton")
print(f"    - Intermediate models for debugging")
print(f"\n  ⚠️  Next steps:")
print(f"    1. Check generated IPs in: {output_dir}/*/ip/")
print(f"    2. Use Vivado to integrate IPs into block design")
print(f"    3. Generate bitstream manually")

✓ Build configuration created (IP generation only)
  Target: xck26-sfvc784-2LV-c @ 200 MHz
  Build steps: 12

  Will generate:
    - Hardware IP cores for all layers
    - PYNQ driver skeleton
    - Intermediate models for debugging

  ⚠️  Next steps:
    1. Check generated IPs in: ../build/*/ip/
    2. Use Vivado to integrate IPs into block design
    3. Generate bitstream manually


In [6]:
# Cell 5: Run the Dataflow Build

import time

print("="*70)
print("STARTING FINN DATAFLOW BUILD FOR KRIA KV260")
print("="*70)
print(f"\nBuild started: {time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Model: {model_path}")
print(f"Output: {output_dir}")
print("\n⚠️  This process will take 2-6 hours depending on model complexity.\n")

start_time = time.time()

try:
    # Run the build
    build.build_dataflow_cfg(model_path, cfg)
    
    elapsed = (time.time() - start_time) / 60
    
    print("\n" + "="*70)
    print("✓ BUILD COMPLETED SUCCESSFULLY")
    print("="*70)
    print(f"\n  Build time: {elapsed:.1f} minutes")
    print(f"  Output directory: {output_dir}")
    print(f"\n  Generated files:")
    print(f"    - Bitstream: Look for .bit file in output_dir")
    print(f"    - Driver: Look for driver/ subdirectory")
    print(f"    - Reports: Look for report/ subdirectory")
    
except Exception as e:
    elapsed = (time.time() - start_time) / 60
    
    print("\n" + "="*70)
    print("✗ BUILD FAILED")
    print("="*70)
    print(f"\n  Failed after: {elapsed:.1f} minutes")
    print(f"  Error: {e}")
    print("\n  Full traceback:")
    import traceback
    traceback.print_exc()
    print(f"\n  Check detailed logs in: {output_dir}/build_dataflow.log")

STARTING FINN DATAFLOW BUILD FOR KRIA KV260

Build started: 2025-12-04 01:26:05
Model: ../../finn_build/ellipse_qonnx_streamlined_fixed.qonnx
Output: ../build

⚠️  This process will take 2-6 hours depending on model complexity.

Building dataflow accelerator from ../../finn_build/ellipse_qonnx_streamlined_fixed.qonnx
Intermediate outputs will be generated in /home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn-kria-kv260-build/build
Final outputs will be generated in ../build
Build log is at ../build/build_dataflow.log
Running step: step_qonnx_to_finn [1/12]
Running step: step_tidy_up [2/12]
Running step: step_streamline [3/12]


/home/hritik/miniconda3/envs/ellipse-finn/lib/python3.9/site-packages/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")
/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn/src/finn/transformation/fpgadataflow/convert_to_hw_layers.py:63: UserWarning: Im2Col_0 : Input is not int. Can't infer ConvInpGen.
  warnings.warn("%s : Input is not int. Can't infer ConvInpGen." % n.name)
/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn/src/finn/transformation/fpgadataflow/convert_to_hw_layers.py:63: UserWarning: Im2Col_1 : Input is not int. Can't infer ConvInpGen.
  warnings.warn("%s : Input is not int. Can't infer ConvInpGen." % n.name)
/home/hritik/Desktop/Hritik/Project/ellipse-regression-project/finn/src/finn/transformation/fpgadataflow/convert_to_hw_layers.py:63: UserWarning: Im2Col_2 : Input is not int. Can't infer ConvInpGen.
  warnings.warn("%s : Input is not int. Can't infer

Running step: step_convert_to_hw [4/12]
Running step: step_convert_non_dataflow_nodes [5/12]

[Custom] Converting non-dataflow nodes...
  Attempted to absorb Transpose nodes
  Found 8 Transpose node(s) to convert
    Converting Transpose_0 with perm=[0, 2, 3, 1]
      → Converted to StreamingFIFO (shape=[0, 1, 20, 20], dtype=FLOAT32)
    Converting Transpose_1 with perm=[0, 3, 1, 2]
      → Converted to StreamingFIFO (shape=[1, 20, 20, 32], dtype=FLOAT32)
    Converting Transpose_2 with perm=[0, 2, 3, 1]
      → Converted to StreamingFIFO (shape=[1, 32, 20, 20], dtype=FLOAT32)
    Converting Transpose_3 with perm=[0, 3, 1, 2]
      → Converted to StreamingFIFO (shape=[1, 10, 10, 64], dtype=FLOAT32)
    Converting Transpose_4 with perm=[0, 2, 3, 1]
      → Converted to StreamingFIFO (shape=[1, 64, 10, 10], dtype=FLOAT32)
    Converting Transpose_5 with perm=[0, 3, 1, 2]
      → Converted to StreamingFIFO (shape=[1, 5, 5, 128], dtype=FLOAT32)
    Converting Transpose_6 with perm=[0, 2, 3

In [7]:
# Cell 5.5: Locate Generated IP Files

import os
from pathlib import Path

print("="*70)
print("LOCATING GENERATED IP FILES")
print("="*70)

build_path = Path(output_dir)

# 1. Check for stitched IP directory
stitched_ip_dir = build_path / "stitched_ip"
if stitched_ip_dir.exists():
    print(f"\n✓ Found stitched_ip directory:")
    print(f"  {stitched_ip_dir}")
    
    # Check for IP subdirectory
    ip_dir = stitched_ip_dir / "ip"
    if ip_dir.exists():
        print(f"\n✓ Found IP directory:")
        print(f"  {ip_dir}")
        
        # List contents
        ip_files = list(ip_dir.iterdir())
        if ip_files:
            print(f"\n  IP files ({len(ip_files)}):")
            for f in ip_files[:10]:
                print(f"    - {f.name}")
            if len(ip_files) > 10:
                print(f"    ... and {len(ip_files) - 10} more")
        else:
            print("  ⚠️ IP directory is empty!")
    else:
        print(f"\n✗ IP subdirectory not found at: {ip_dir}")
        print("\n  Contents of stitched_ip:")
        for item in stitched_ip_dir.iterdir():
            print(f"    - {item.name}")
    
    # Check for Vivado project
    vivado_proj = stitched_ip_dir / "finn_vivado_stitch_proj.xpr"
    if vivado_proj.exists():
        print(f"\n✓ Found Vivado project:")
        print(f"  {vivado_proj}")
    else:
        print(f"\n✗ Vivado project not found")
        
else:
    print(f"\n✗ stitched_ip directory not found at: {stitched_ip_dir}")
    print("\n  This means step_create_stitched_ip was not run.")
    print("  Checking for intermediate IP directories...")

# 2. Check for individual layer IP directories
print("\n" + "-"*70)
print("CHECKING FOR INDIVIDUAL LAYER IPs")
print("-"*70)

intermediate_dir = build_path / "intermediate_models"
if intermediate_dir.exists():
    # Look for directories containing IP generation
    ip_dirs = []
    
    # Check for HLS IP directories
    for item in intermediate_dir.rglob("*"):
        if item.is_dir() and ("ip" in item.name or "ipgen" in item.name):
            ip_dirs.append(item)
    
    if ip_dirs:
        print(f"\n✓ Found {len(ip_dirs)} IP-related directories:")
        for ip_dir in ip_dirs[:10]:
            rel_path = ip_dir.relative_to(build_path)
            print(f"  {rel_path}")
            # Check if it has actual IP files
            ip_contents = list(ip_dir.glob("*.zip")) + list(ip_dir.glob("component.xml"))
            if ip_contents:
                print(f"    → Contains IP: {[f.name for f in ip_contents[:3]]}")
        
        if len(ip_dirs) > 10:
            print(f"  ... and {len(ip_dirs) - 10} more")
    else:
        print("\n✗ No individual IP directories found")
else:
    print(f"\n✗ intermediate_models directory not found")

# 3. Check build configuration
print("\n" + "-"*70)
print("BUILD CONFIGURATION CHECK")
print("-"*70)

print(f"\nGenerate outputs requested:")
for output_type in cfg.generate_outputs:
    print(f"  - {output_type.name}")

print(f"\nBuild steps executed:")
for i, step in enumerate(cfg.steps, 1):
    step_name = step.__name__ if hasattr(step, '__name__') else str(step)
    print(f"  {i}. {step_name}")

# Check if step_create_stitched_ip is in the list
has_stitch_step = any('stitch' in str(step).lower() for step in cfg.steps)
if not has_stitch_step:
    print("\n⚠️  WARNING: step_create_stitched_ip is NOT in build steps!")
    print("   This is why stitched_ip directory doesn't exist.")

print("\n" + "="*70)

LOCATING GENERATED IP FILES

✗ stitched_ip directory not found at: ../build/stitched_ip

  This means step_create_stitched_ip was not run.
  Checking for intermediate IP directories...

----------------------------------------------------------------------
CHECKING FOR INDIVIDUAL LAYER IPs
----------------------------------------------------------------------

✗ No individual IP directories found

----------------------------------------------------------------------
BUILD CONFIGURATION CHECK
----------------------------------------------------------------------

Generate outputs requested:
  - RTLSIM_PERFORMANCE

Build steps executed:
  1. step_qonnx_to_finn
  2. step_tidy_up
  3. step_streamline
  4. step_convert_to_hw
  5. step_convert_non_dataflow_nodes
  6. step_target_fps_parallelization
  7. step_apply_folding_config
  8. step_minimize_bit_width
  9. step_hw_codegen
  10. step_hw_ipgen
  11. step_set_fifo_depths_custom
  12. step_make_pynq_driver

⚠️  WARNING: step_create_stitch

In [8]:
# Cell 6: Analyze Build Results

import os
import json
from pathlib import Path

print("="*70)
print("BUILD RESULTS ANALYSIS")
print("="*70)

if not os.path.exists(output_dir):
    print("✗ Build directory not found!")
else:
    print(f"\n✓ Build directory: {output_dir}\n")
    
    # Find key files
    build_path = Path(output_dir)
    
    # 1. Bitstream
    bitfiles = list(build_path.rglob("*.bit"))
    if bitfiles:
        print(f"✓ Bitstream Files ({len(bitfiles)}):")
        for bf in bitfiles:
            size_mb = bf.stat().st_size / (1024*1024)
            print(f"    {bf.relative_to(build_path)} ({size_mb:.2f} MB)")
    else:
        print("⚠ No bitstream (.bit) files found")
    
    # 2. Hardware handoff
    hwh_files = list(build_path.rglob("*.hwh"))
    if hwh_files:
        print(f"\n✓ Hardware Handoff Files ({len(hwh_files)}):")
        for hwh in hwh_files:
            print(f"    {hwh.relative_to(build_path)}")
    
    # 3. Driver
    driver_files = list(build_path.rglob("driver.py"))
    if driver_files:
        print(f"\n✓ Driver Files ({len(driver_files)}):")
        for df in driver_files:
            print(f"    {df.relative_to(build_path)}")
    
    # 4. Reports
    report_files = list(build_path.rglob("*estimate*.json"))
    if report_files:
        print(f"\n✓ Resource Estimates ({len(report_files)}):")
        for rf in sorted(report_files)[:3]:
            print(f"    {rf.relative_to(build_path)}")
            try:
                with open(rf) as f:
                    data = json.load(f)
                    if 'total' in data:
                        print(f"      LUT: {data['total'].get('LUT', 'N/A')}")
                        print(f"      FF: {data['total'].get('FF', 'N/A')}")
                        print(f"      BRAM: {data['total'].get('BRAM_18K', 'N/A')}")
                        print(f"      DSP: {data['total'].get('DSP', 'N/A')}")
            except:
                pass
    
    # 5. Deployment package
    deploy_dirs = [d for d in build_path.rglob("deploy*") if d.is_dir()]
    if deploy_dirs:
        print(f"\n✓ Deployment Package:")
        for dd in deploy_dirs:
            print(f"    {dd.relative_to(build_path)}/")
            deploy_files = list(dd.iterdir())[:5]
            for df in deploy_files:
                print(f"      - {df.name}")
    
    # 6. Intermediate models
    intermediate_models = list(build_path.rglob("intermediate_models/*.onnx"))
    if intermediate_models:
        print(f"\n✓ Intermediate Models ({len(intermediate_models)}):")
        for im in sorted(intermediate_models)[:5]:
            print(f"    {im.name}")
        if len(intermediate_models) > 5:
            print(f"    ... and {len(intermediate_models) - 5} more")
    
    print("\n" + "="*70)

BUILD RESULTS ANALYSIS

✓ Build directory: ../build

⚠ No bitstream (.bit) files found

✓ Resource Estimates (1):
    report/estimate_layer_resources_hls.json

✓ Intermediate Models (12):
    step_apply_folding_config.onnx
    step_convert_non_dataflow_nodes.onnx
    step_convert_to_hw.onnx
    step_hw_codegen.onnx
    step_hw_ipgen.onnx
    ... and 7 more



In [ ]:
# Cell 7: Create Deployment Package for KV260

import shutil
from pathlib import Path
import time

print("="*70)
print("CREATING DEPLOYMENT PACKAGE FOR KV260")
print("="*70)

# Create deployment directory
deploy_output = Path(output_dir) / "kv260_deployment"
deploy_output.mkdir(exist_ok=True)

print(f"\n✓ Deployment directory: {deploy_output}\n")

build_path = Path(output_dir)
files_copied = 0

# Copy bitstream
bitfiles = list(build_path.rglob("*.bit"))
if bitfiles:
    shutil.copy(bitfiles[0], deploy_output / "finn_accel.bit")
    print(f"  ✓ Copied: finn_accel.bit")
    files_copied += 1
else:
    print(f"  ⚠ No bitstream found")

# Copy hardware handoff
hwh_files = list(build_path.rglob("*.hwh"))
if hwh_files:
    shutil.copy(hwh_files[0], deploy_output / "finn_accel.hwh")
    print(f"  ✓ Copied: finn_accel.hwh")
    files_copied += 1
else:
    print(f"  ⚠ No .hwh file found")

# Copy driver
driver_files = list(build_path.rglob("driver.py"))
if driver_files:
    shutil.copy(driver_files[0], deploy_output / "driver.py")
    print(f"  ✓ Copied: driver.py")
    files_copied += 1
else:
    print(f"  ⚠ No driver found")

# Copy reports
report_files = list(build_path.rglob("*estimate*.json"))
if report_files:
    shutil.copy(report_files[0], deploy_output / "resource_estimate.json")
    print(f"  ✓ Copied: resource_estimate.json")
    files_copied += 1

# Create README
readme = f"""# Ellipse Regression FINN Accelerator - Kria KV260 Deployment

## Overview

This package contains the FINN-generated accelerator for ellipse regression on Kria KV260.

## Files

- `finn_accel.bit` - Bitstream for Kria KV260
- `finn_accel.hwh` - Hardware handoff file  
- `driver.py` - Python driver code
- `resource_estimate.json` - Resource utilization report

## Deployment to KV260

### 1. Copy Files to KV260

```bash
scp -r {deploy_output.name} xilinx@<kv260-ip>:~/
```

### 2. On KV260, Load the Overlay

```python
from pynq import Overlay

# Load the accelerator
overlay = Overlay("finn_accel.bit")

# Access the accelerator (adjust based on your driver)
accel = overlay.finn_accel_0
```

### 3. Run Inference

```python
import numpy as np

# Prepare input data
input_data = np.random.randint(0, 255, size=(1, 3, 224, 224), dtype=np.uint8)

# Run inference
output = accel.execute(input_data)

print("Ellipse parameters:", output)
```

## Build Information

- **Target**: Kria KV260 (xck26-sfvc784-2LV-c)
- **Clock**: 200 MHz
- **Build date**: {time.strftime('%Y-%m-%d %H:%M:%S')}
- **FINN version**: Check build logs

## Resource Usage

See `resource_estimate.json` for detailed resource utilization.

## Troubleshooting

1. **Overlay won't load**: Check that bitstream matches FPGA part
2. **Driver import error**: Ensure PYNQ is installed on KV260
3. **Execution hangs**: Check FIFO depths and DMA configuration

## References

- FINN Documentation: https://finn.readthedocs.io
- Kria KV260: https://www.xilinx.com/products/som/kria/kv260-vision-starter-kit.html
- PYNQ: http://www.pynq.io
"""

with open(deploy_output / "README.md", 'w') as f:
    f.write(readme)
print(f"  ✓ Created: README.md")
files_copied += 1

# Create a simple test script
test_script = """#!/usr/bin/env python3
\"\"\"
Simple test script for FINN accelerator on Kria KV260
\"\"\"

import numpy as np
from pynq import Overlay
import time

print("Loading FINN accelerator overlay...")
overlay = Overlay("finn_accel.bit")
print("✓ Overlay loaded")

# Get accelerator handle
# Note: Adjust 'finn_accel_0' to match your actual IP name
try:
    accel = overlay.finn_accel_0
    print("✓ Accelerator found")
except:
    print("✗ Could not find accelerator IP")
    print("Available IPs:", dir(overlay))
    exit(1)

# Prepare test input
print("\\nPreparing test input...")
input_shape = (1, 3, 224, 224)  # Adjust to your model input
input_data = np.random.randint(0, 255, size=input_shape, dtype=np.uint8)
print(f"  Input shape: {input_shape}")

# Run inference
print("\\nRunning inference...")
start = time.time()
output = accel.execute(input_data)
elapsed = time.time() - start

print(f"✓ Inference complete in {elapsed*1000:.2f} ms")
print(f"  Output shape: {output.shape}")
print(f"  Output: {output}")

print("\\n✓ Test completed successfully!")
"""

with open(deploy_output / "test_kv260.py", 'w') as f:
    f.write(test_script)
print(f"  ✓ Created: test_kv260.py")
files_copied += 1

print(f"\n✓ Deployment package complete: {files_copied} files")
print(f"\n📦 Ready to deploy:")
print(f"   scp -r {deploy_output.name} xilinx@<kv260-ip>:~/")

print("\n" + "="*70)

CREATING DEPLOYMENT PACKAGE FOR KV260

✓ Deployment directory: ../build/kv260_deployment

  ⚠ No bitstream found
  ⚠ No .hwh file found
  ⚠ No driver found
  ✓ Copied: resource_estimate.json
  ✓ Created: README.md
  ✓ Created: test_kv260.py

✓ Deployment package complete: 3 files

📦 Ready to deploy:
   scp -r kv260_deployment xilinx@<kv260-ip>:~/

📝 See README.md for deployment instructions

